In [16]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import time

In [5]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [7]:
load_dotenv()

True

In [8]:
api_key = os.environ.get("OpenAI_API_KEY")

In [78]:
client = OpenAI(api_key=api_key)
assistant = client.beta.assistants.create(
    name="Recipe Bot",
    instructions="""
        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:

        Selected ingredients: A list of ingredients chosen by the user.
        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).

        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:

        - A title describing the dish.
        - A brief summary of the dish.
        - A list of ingredients (including the selected ones).
        - Step-by-step cooking instructions.

        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.
    
    
    """,
    model="gpt-4o-mini"
)

show_json(assistant)


{'id': 'asst_bCEr907TRU59ITBx2Oof8Dse',
 'created_at': 1731846566,
 'description': None,
 'instructions': "\n        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:\n\n        Selected ingredients: A list of ingredients chosen by the user.\n        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).\n\n        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:\n\n        - A title describing the dish.\n        - A brief summary of the dish.\n        - A list of ingredients (including the selected ones).\n        - Step-by-step cooking instructions.\n\n        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.\n    \n    \n    ",
 'metadata': {},
 'model': 'gpt

In [86]:
ASSISTANT_ID = assistant.id
print(f"Assistants ID:\n{ASSISTANT_ID}")

Assistants ID:
asst_bCEr907TRU59ITBx2Oof8Dse


In [50]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )
    return run

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        time.sleep(0.5)
    return run

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(ASSISTANT_ID, thread, user_input)
    return thread, run

def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")
    print("-" * 30)

In [ ]:
user_input = "양파, 감자, 파스타면, 양송이, 간장, 김치, 대파, 삼겹살, 간장"

In [84]:
thread1, run1 = create_thread_and_run(user_input)

In [85]:
print_message(get_response(thread1))

[USER]
양파, 감자, 파스타면, 양송이, 간장, 김치, 대파, 삼겹살, 간장

[ASSISTANT]
### 레시피 1: 삼겹살 감자 파스타 볶음

**요약:** 삼겹살과 감자, 파스타면을 간장으로 맛을 낸 볶음 요리입니다. 깊은 풍미와 쫄깃한 식감이 어우러져 식사로 손색이 없습니다.

**재료:**
- 삼겹살 200g
- 감자 2개 (중간 크기)
- 파스타면 150g
- 양파 1개
- 양송이 100g
- 대파 1대
- 간장 3큰술
- 후춧가루 약간
- 식용유 2큰술

**조리 방법:**
1. 감자는 껍질을 벗기고 1cm 크기로 썬 후, 찬물에 담가 전분을 빼줍니다.
2. 파스타면은 끓는 물에 8~10분간 삶아준 후, 체에 받쳐 물기를 제거합니다.
3. 팬에 식용유를 두르고 중불에서 삼겹살을 넣어 노릇하게 볶아줍니다.
4. 삼겹살이 익으면 양파와 대파를 넣고 볶다가, 감자를 추가해 황금빛으로 튀길 때까지 약 5분 볶습니다.
5. 양송이를 넣고 2-3분 더 볶다가 간장을 넣어 볶아줍니다.
6. 마지막으로 삶은 파스타면을 넣고 고루 섞어 후춧가루로 간을 맞춘 뒤, 불을 끄고 접시에 담아냅니다.

---

### 레시피 2: 간장 양송이 김치 파스타

**요약:** 간장으로 양송이와 김치를 조리하여 독특한 맛을 냈습니다. 파스타와 결합하여 간단하면서도 특별한 한 끼로 즐길 수 있습니다.

**재료:**
- 파스타면 150g
- 양송이 100g
- 김치 100g
- 양파 1개
- 대파 1대
- 간장 2큰술
- 올리브유 1큰술
- 소금, 후춧가루 약간

**조리 방법:**
1. 파스타면을 끓는 물에서 8~10분 삶아 체에 받쳐두세요.
2. 팬에 올리브유를 두르고 chopped 양파와 대파를 중불에서 볶아줍니다.
3. 양파가 투명해지면 양송이를 넣고 3분 더 볶습니다.
4. 김치를 넣고 간장도 추가해 5분간 더 조리합니다.
5. 삶은 파스타면을 넣고 재료와 잘 섞이도록 볶은 후, 소금과 후춧가루로 간을 합니다.
6. 그릇에 담아내고 원한다면 통깨를 뿌려 장식하세요.
